In [1]:
# Import packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import matplotlib as mpl
import pandas as pd
import os
import sys
import argparse
import re
import seaborn as sns
from scipy import stats
# IMport cv2
import cv2
from skimage import io
from skimage import color
from skimage.segmentation import watershed
import scipy.io
import scipy.ndimage
import h5py

from scipy.ndimage import label, generate_binary_structure
from skimage.measure import regionprops
from skimage.io import imread, imshow

from skimage import measure, morphology
import math
from h5py_to_dict import h5py_to_dict
from getPrmDflt import getPrmDflt
from bwareaopen import bwareaopen
from get_con_seg import get_con_seg
from edges import edges


# Get the path to the parent directory
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
# # Append the parent directory to the system path
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
# Append the parent directory to the system path
parent_parent_dir = os.path.abspath(os.path.join(parent_dir, os.pardir))
if parent_parent_dir not in sys.path:
    sys.path.append(parent_parent_dir)
# Add to path parent_parent_dir/MLVcode
sys.path.append(os.path.join(parent_parent_dir, 'MLVcode'))
print("The current working directory is:")
print(os.getcwd())
print("The parent directory is:")
print(parent_dir)
print("The parent parent directory is:")
print(parent_parent_dir)
from mergeLineSegments import mergeLineSegments

The current working directory is:
/Users/aravind/Documents/BWL_github/July_PyMLV/pyMLV/demos/getLineDrawing
The parent directory is:
/Users/aravind/Documents/BWL_github/July_PyMLV/pyMLV/demos
The parent parent directory is:
/Users/aravind/Documents/BWL_github/July_PyMLV/pyMLV


# spAffinities

In [3]:
# Load the data required - S_spAffinities.mat, E_spAffinities.mat, 
# and segs_spAffinities.mat using scipy.io.loadmat

S = scipy.io.loadmat('Data_spAffinities/S_spAffinities.mat')['S_spAffinities']
E = scipy.io.loadmat('Data_spAffinities/E_spAffinities.mat')['E_spAffinities']
segs = scipy.io.loadmat('Data_spAffinities/segs_spAffinities.mat')['segs_orig']


In [4]:
# def affinities(S, h, w, E, segs, dims, nThreads):
#     g, _, h1, w1, nTreesEval = dims
#     stride = int(float(h) / float(h1) + 0.5)
#     m = np.max(S) + 1
#     A = np.zeros((m - 1) * (m - 1), dtype=np.float32)
#     wts = 1 / (1 + np.exp((E - 0.05) * 50.0))
#     print("WEIGHTS")
#     print(wts[0:7, 0:7])
#     Sn = np.zeros((m, m), dtype=np.float32)
#     Sd = np.zeros((m, m), dtype=np.float32)
    
#     def compute_local_affinities(x, y):
#         r = g // 2
#         x0 = max(-r, -x)
#         x1 = min(r, w - x)
#         y0 = max(-r, -y)
#         y1 = min(r, h - y)
        
#         lookup = np.zeros(m, dtype=np.uint32)
#         lbls1 = np.zeros((g, g), dtype=np.uint32)
#         wts1 = np.zeros(m * 25, dtype=np.float32)
        
#         lookup[0] = S[x + x0, y + y0]
#         wts1[0] = 0
#         j = 0
#         m1 = 1
#         nTreesConst = 0
        
#         for xi in range(x0, x1):
#             for yi in range(y0, y1):
#                 lbl = S[x + xi, y + yi]
#                 if lbl == lookup[j]:
#                     i = j
#                 else:
#                     indices = np.where(lookup[:m1] == lbl)[0]
#                     if len(indices) == 0:
#                         lookup[m1] = lbl
#                         wts1[m1] = 0
#                         i = m1
#                         m1 += 1
#                     else:
#                         i = indices[0]
#                 lbls1[xi + r, yi + r] = i
#                 j = i
#                 wts1[i] += wts[x + xi, y + yi]
        
#         for t in range(nTreesEval):
#             ind = (y // stride) + (x // stride) * h1 + t * h1 * w1
#             seg = segs[:, :, int(y / stride), int(x / stride), t].reshape((g, g))
#             s1 = np.max(seg)
#             if s1 == 0:
#                 nTreesConst += 1
#                 continue
#             for i in range(m1, m1 * (s1 + 1)):
#                 wts1[i] = 0
#             for xi in range(x0, x1):
#                 for yi in range(y0, y1):
#                     i = (xi + r) * g + (yi + r)
#                     wts1[lbls1[xi + r, yi + r] + (seg[xi + r, yi + r] + 1) * m1] += wts[x + xi, y + yi]
#             for s in range(1, s1 + 1):
#                 for i in range(m1):
#                     for j in range(m1):
#                         Sn[lookup[i], lookup[j]] += wts1[i + s * m1] * wts1[j + s * m1]
        
#         if nTreesConst:
#             for i in range(m1):
#                 for j in range(m1):
#                     Sn[lookup[i], lookup[j]] += wts1[i] * wts1[j] * nTreesConst
        
#         for i in range(m1):
#             for j in range(m1):
#                 Sd[lookup[i], lookup[j]] += wts1[i] * wts1[j] * nTreesEval
        
#         return Sn, Sd
    
#     for x in range(0, w, stride):
#         for y in range(0, h, stride):
#             Sn, Sd = compute_local_affinities(x, y)
#             break
#         break
    
#     print("SN")
#     print(Sn)
#     print("SD")
#     print(Sd)
#     return Sn, Sd
#     for s in range(1, m):
#         for t in range(1, m):
#             if Sd[s, t] != 0:
#                 d = Sn[s, s] / Sd[s, s] / 2 + Sn[t, t] / Sd[t, t] / 2 - Sn[s, t] / Sd[s, t]
#                 A[(s - 1) * (m - 1) + (t - 1)] = 1 - max(0.0, d)
    
#     return A

# # Example usage
# S = scipy.io.loadmat('Data_spAffinities/S_spAffinities.mat')['S_spAffinities']
# E = scipy.io.loadmat('Data_spAffinities/E_spAffinities.mat')['E_spAffinities']
# segs = scipy.io.loadmat('Data_spAffinities/segs_spAffinities.mat')['segs_orig']
# h, w = S.shape
# dims = segs.shape
# nThreads = 4

# # A = affinities(S, h, w, E, segs, dims, nThreads)
# # print(A)
# Sn, Sd = affinities(S, h, w, E, segs, dims, nThreads)

In [218]:
# import numpy as np
# from tqdm import tqdm

# def lin_ind_to_sub(ind, shape):
#     return np.unravel_index(ind, shape)

# def affinities(S, h, w, E, segs, dims, nThreads):
#     g, _, h1, w1, nTreesEval = dims
#     stride = int(float(h) / float(h1) + 0.5)
#     m = np.max(S) + 1
#     A = np.zeros((m - 1) * (m - 1), dtype=np.float32)
#     wts = 1 / (1 + np.exp((E - 0.05) * 50.0))
#     Sn = np.zeros((m, m), dtype=np.float32)
#     Sd = np.zeros((m, m), dtype=np.float32)
    
#     def compute_local_affinities(x, y, printing=False):
#         r = g // 2
#         x0 = max(-r, -x)
#         x1 = min(r, w - x)
#         y0 = max(-r, -y)
#         y1 = min(r, h - y)
        
#         lookup = np.zeros(m, dtype=np.uint32)
#         lbls1 = np.zeros((g, g), dtype=np.uint32)
#         wts1 = np.zeros(m * 25, dtype=np.float32)
#         # lookup[0] = S[lin_ind_to_sub((x+x0)*h + (y+y0), S.shape)]
#         lookup[0] = S[y+y0, x+x0]
#         wts1[0] = 0
#         j = 0
#         m1 = 1
#         nTreesConst = 0
#         if printing:
#             print("x, y: ", x, y)
#             print("x0, x1, y0, y1: ", x0, x1, y0, y1)
#             print("lookup: ", lookup[0])
#             print("j: ", j)
#             print("m1: ", m1)
#             print("nTreesConst: ", nTreesConst)
        
        
#         for xi in range(x0, x1):
#             for yi in range(y0, y1):
#                 if printing:
#                     print("xi, yi?: ", xi, yi)
#                 # print("xi, yi?: ", xi, yi)
#                 s_ind = (x+xi)*h + (y+yi)
#                 lbl = S[y+yi, x+xi]
#                 if printing:
#                     print("s_ind: ", s_ind, "lbl: ", lbl, "lookup[j]: ", lookup[j], "j: ", j)
#                 if lbl == lookup[j]:
#                     i = j
#                     if printing:
#                         print("----------i: ", i)
#                 else:
#                     flag_break = False
#                     for i in range(m1):
#                         if lookup[i] == lbl:
#                             flag_break = True
#                             break
#                     if flag_break == False:
#                         i = i + 1
#                     if i == m1:
#                         lookup[m1] = lbl
#                         wts1[m1] = 0
#                         m1 += 1
#                 lbls1_ind = (xi + r)*g + (yi + r)
#                 lbls1[xi+r, yi+r] = i
#                 j = i
#                 wts_ind = (x+xi)*h + y+yi
#                 if printing:
#                     print("wts_ind: ", wts_ind, "i: ", i)
#                 wts1[i] += wts[y+yi, x+xi]
        
#         # print("lbls1: ", lbls1)
#         # print("wts1: ", wts1[0:5])
#         # # Print non-zero values of wts1
#         # print("Non-zero values of wts1: ", np.nonzero(wts1))
#         # print("lbls1 shape: ", lbls1.shape)
#         # print("wts1 shape: ", wts1.shape)
#         for t in range(nTreesEval):
#             ind = (y // stride) + (x // stride) * h1 + t * h1 * w1
            
#             seg = segs[:, :, int(y / stride), int(x / stride), t].reshape((g, g))
#             # if printing:
#             #     print("seg: ", seg) 
#             s1 = np.max(seg)
#             # print("s1: ", s1)
#             if s1 == 0:
#                 nTreesConst += 1
#                 print("Continuing for nTreesConst: ", nTreesConst, "x, y: ", x, y)
#                 continue
#             for i in range(m1, m1 * (s1 + 1)):
#                 wts1[i] = 0
                
#             for xi in range(x0, x1):
#                 for yi in range(y0, y1):
#                     wts1[lbls1[yi+r, xi+r] + (seg[yi+r, xi+r] + 1) * m1] += wts[y+yi, x+xi]
#             for s in range(1, s1 + 2):
#                 print("s: ", s, "s1: ", s1)
#                 for i in range(m1):
#                     for j in range(m1):
#                         Sn[lookup[i], lookup[j]] += wts1[i + s * m1] * wts1[j + s * m1]
        
#         if nTreesConst!=0:
#             print("nTreesConst: ", nTreesConst, "x, y: ", x, y, "nTreesEval: ", nTreesEval)
#             for i in range(m1):
#                 for j in range(m1):
#                     Sn[lookup[i], lookup[j]] += wts1[i] * wts1[j] * nTreesConst
        
#         for i in range(m1):
#             for j in range(m1):
#                 Sd[lookup[i], lookup[j]] += wts1[i] * wts1[j] * nTreesEval
    
#     for x in tqdm(range(0, w, stride)):
#         for y in range(0, h, stride):
#             # print(x, y)
#             if x == 0 and y < 10:
#                 print(x, y)
#                 compute_local_affinities(x, y, printing=False)
#             else:
#                 compute_local_affinities(x, y)
#             if x == 0 and y == 2:
#                 break
#         if x == 0:
#             break
#     return Sn, Sd
#     # for s in range(1, m):
#     #     for t in range(1, m):
#     #         if Sd[s * m + t] != 0:
#     #             d = Sn[s * m + s] / Sd[s * m + s] / 2 + Sn[t * m + t] / Sd[t * m + t] / 2 - Sn[s * m + t] / Sd[s * m + t]
#     #             A[(s - 1) * (m - 1) + (t - 1)] = 1 - max(0.0, d)
    
#     # return A
    
# # Example usage
# S = scipy.io.loadmat('Data_spAffinities/S_spAffinities.mat')['S_spAffinities']
# E = scipy.io.loadmat('Data_spAffinities/E_spAffinities.mat')['E_spAffinities']
# segs = scipy.io.loadmat('Data_spAffinities/segs_spAffinities.mat')['segs_orig']
# h, w = S.shape
# dims = segs.shape
# nThreads = 4

# # A = affinities(S, h, w, E, segs, dims, nThreads)
# # print(A)
# Sn, Sd = affinities(S, h, w, E, segs, dims, nThreads)
# Sn

In [219]:
# # Print non-zero values of Sn and Sd
# print("Non-zero indices of Sn")
# print(np.nonzero(Sn))
# print("Non-zero indices of Sd")
# print(np.nonzero(Sd))
# # Print in column-major order
# print("Non-zero indices of Sn in column-major order")
# # print(np.transpose(np.nonzero(Sn)))
# print(len(np.transpose(np.nonzero(Sn))))
# print("Non-zero indices of Sd in column-major order")
# # print(np.transpose(np.nonzero(Sd)))
# print(len(np.transpose(np.nonzero(Sd))))

# # Print linear indices of non-zero values of Sn and Sd like if they were 1D arrays using np.ravel_multi_index((row, col), shape, order='C')
# print("Linear indices of non-zero values of Sn")
# Sd_nonzero = np.nonzero(Sd)
# Sn_nonzero = np.nonzero(Sn)
# print(np.ravel_multi_index(Sd_nonzero, Sd.shape)+1)
# print(np.ravel_multi_index(Sn_nonzero, Sn.shape)+1)


In [220]:
# # Disp size of S, E, segs
# print(S.shape)
# print(E.shape)
# print(segs.shape)
# print(h, w)

# spAffinities - edges - works

In [2]:
S = scipy.io.loadmat('Data_spAffinities/S_spAffinities.mat')['S_spAffinities']
A = scipy.io.loadmat('Data_spAffinities/A_spAffinities.mat')['A_spAffinities']
h, w = S.shape
E = np.zeros((h, w), dtype=np.float32)
updated_E = edges(E, S, h, w, A)
print("Updated E")
print(updated_E[:10, :5])
print("\n")
E_edges_MATLAB = scipy.io.loadmat('Data_spAffinities/updated_E.mat')['updated_E']
print("E_edges_MATLAB")
print(E_edges_MATLAB[:10,:5])
# Check ig updated_E is equal to E_edges_MATLAB
if np.allclose(updated_E, E_edges_MATLAB):
    print("updated_E is equal to E_edges_MATLAB")
else:
    print("updated_E is not equal to E_edges_MATLAB")

Updated E
[[0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.08511275]
 [0.         0.         0.         0.         0.08511275]
 [0.         0.         0.09506911 0.09506911 0.09506911]
 [0.09506911 0.09506911 0.09506911 0.         0.03093755]
 [0.         0.         0.         0.         0.03093755]
 [0.         0.         0.         0.         0.        ]]


E_edges_MATLAB
[[0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.08511275]
 [0.         0.         0.         0.         0.08511275]
 [0.         0.         0.09506911 0.0950691

# computeUcm

In [3]:
import scipy.ndimage

def upsampleEdges(E0):
    # upsample E by factor of two while mostly keeping edges thin
    h, w = E0.shape
    h = 2 * h
    w = 2 * w
    E = np.zeros((h, w))
    E[0:h - 1:2, 0:w - 1:2] = E0
    E[0:h - 1:2, 1:w - 2:2] = np.minimum(E0[:, 0:-1], E0[:, 1:])
    E[h - 1, :] = E[h - 2, :]
    E[1:h - 2:2, 0:w - 1:2] = np.minimum(E0[0:-1, :], E0[1:, :])

    # Remove single pixel segments created by thick edges in E0 (2x2 blocks)
    A = np.ones((2, 2)) / 4
    A = scipy.ndimage.convolve((E0 > 0).astype(np.float32), A) == 1
    xs, ys = np.where(A)

    for i in range(len(xs)):
        x = (xs[i] - 1) * 2
        y = (ys[i] - 1) * 2
        es = np.ones((2, 4)) + 1
        
        if x > 2 and y > 2:
            es[:, 0] = [E[x - 2, y - 1], E[x - 1, y - 2]]
        if x < h - 2 and y > 2:
            es[:, 1] = [E[x + 2, y - 1], E[x + 1, y - 2]]
        if x < h - 2 and y < w - 2:
            es[:, 2] = [E[x + 2, y + 1], E[x + 1, y + 2]]
        if x > 2 and y < w - 2:
            es[:, 3] = [E[x - 2, y + 1], E[x - 1, y + 2]]
        
        max_values = np.max(es, axis=0)
        j = np.argmax(max_values)
        e = max_values[j]
        
        if j == 0 or j == 3:
            x1 = x - 1
        else:
            x1 = x + 1
        if j == 0 or j == 1:
            y1 = y - 1
        else:
            y1 = y + 1
        
        E[x, y1] = e
        E[x1, y] = e
        E[x1, y1] = e
        
        if es[0, j] < es[1, j]:
            E[x, y1] = 0
        else:
            E[x1, y] = 0
    
    return E

def computeUcm(E):
    E = upsampleEdges(E)
    print("Upsample edges done")
    print(E[:28, 1:7])
    return E
    S = measure.label(E == 0, connectivity=2) - 1
    S = S[1::2, 1::2]
    S[-1, :] = S[-2, :]
    S[:, -1] = S[:, -2]
    E = np.pad(E, ((1, 1), (1, 1)), mode='edge')
    U = ucm_mean_pb.compute_ucm(E, S)  # Need to implement this function
    return U
    U = U[0:-2:2, 0:-2:2]
    return U

E_temp = computeUcm(E_edges_MATLAB)

Upsample edges done
[[0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.09506911 0.09506911 0.09506911]
 [0.         0.         0.         0.09506911 0.         0.        ]
 [0.09506911 0

In [28]:
import numpy as np
from scipy.signal import convolve2d

def upsample_edges_temp(E0):
    """
    Upsample edges by a factor of two while keeping edges thin.

    Parameters:
    E0 (numpy.ndarray): Input edge map.

    Returns:
    E (numpy.ndarray): Upsampled edge map.
    """
    h, w = E0.shape
    h *= 2
    w *= 2
    E = np.zeros((h, w))

    # Upsample E0 by factor of two
    E[1:h:2, 1:w:2] = E0
    E[1:h:2, 2:w:2] = np.minimum(E0[:, :-1], E0[:, 1:])
    E[2:h:2, 1:w:2] = np.minimum(E0[:-1, :], E0[1:, :])
    E[2:h:2, 2:w:2] = np.minimum(E0[:-1, :-1], np.minimum(E0[:-1, 1:], np.minimum(E0[1:, :-1], E0[1:, 1:])))
    E[0, :] = E[1, :]
    E[h-1, :] = E[h-2, :]
    E[:, 0] = E[:, 1]
    E[:, w-1] = E[:, w-2]

    # Remove single pixel segments created by thick edges in E0 (2x2 blocks)
    A = np.ones((2, 2)) / 4
    A = convolve2d((E0 > 0).astype(np.float32), A, mode='same') == 1
    xs, ys = np.where(A)

    for i in range(len(xs)):
        x = (xs[i] - 1) * 2
        y = (ys[i] - 1) * 2
        es = np.ones((2, 4)) + 1

        if x > 2 and y > 2:
            es[:, 0] = [E[x-2, y-1], E[x-1, y-2]]
        if x < h-2 and y > 2:
            es[:, 1] = [E[x+2, y-1], E[x+1, y-2]]
        if x < h-2 and y < w-2:
            es[:, 2] = [E[x+2, y+1], E[x+1, y+2]]
        if x > 2 and y < w-2:
            es[:, 3] = [E[x-2, y+1], E[x-1, y+2]]

        e, j = np.min(np.max(es, axis=0), axis=0), np.argmin(np.max(es, axis=0))
        if j == 0 or j == 3:
            x1 = x - 1
        else:
            x1 = x + 1
        if j == 0 or j == 1:
            y1 = y - 1
        else:
            y1 = y + 1

        E[x, y1] = e
        E[x1, y] = e
        E[x1, y1] = e
        if es[0, j] < es[1, j]:
            E[x, y1] = 0
        else:
            E[x1, y] = 0

    E[0, :] = 1
    E[h-1, :] = 1
    E[:, 0] = 1
    E[:, w-1] = 1

    for i in range(1, h-1):
        for j in range(1, w-1):
            if i % 2 == 1:
                E[i, j] = 1 - E[i, j]
            else:
                E[i, j] = E[i, j]

    return E

# Test the function
E0 = np.ones((5, 5))  # Example input
E = upsample_edges_temp(E0)
print(E)

[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 0. 1. 0. 1. 0. 1. 0. 0. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 0. 1. 0. 1. 0. 1. 0. 0. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 0. 1. 0. 1. 0. 1. 0. 0. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 0. 1. 0. 0. 0. 0. 0. 0. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]


In [6]:
from scipy.ndimage import label

def bwlabel(BW, mode=8):
    """
    Label connected components in 2-D binary image.
    
    Parameters:
    BW (array-like): Input 2D binary image.
    mode (int): Connectivity, either 4 or 8 (default is 8).
    
    Returns:
    L (ndarray): Labeled matrix.
    numComponents (int): Number of connected components found in BW.
    """
    if mode not in [4, 8]:
        raise ValueError("Mode must be either 4 or 8")
    
    # Define connectivity structure
    if mode == 4:
        structure = np.array([[0, 1, 0],
                              [1, 1, 1],
                              [0, 1, 0]])
    else:  # mode == 8
        structure = np.ones((3, 3), dtype=int)
    
    # Ensure BW is binary
    BW = (BW != 0)
    
    # Label the connected components
    L, numComponents = label(BW, structure=structure)
    
    return L, numComponents

In [6]:
def temp(E_init):
    S, num = bwlabel(E_init==0, mode=8)
    S = S[1::2, 1::2] - 1
    S[-1, :] = S[-2, :]
    S[:, -1] = S[:, -2]
    E = np.pad(E_init, ((0, 1), (0, 1)), mode='edge')
    import ucm_mean_pb
    U = ucm_mean_pb.compute_ucm(E, S)
    return E, S
E_temp_temp, S_temp_temp = temp(E_temp)
print(E_temp_temp.shape)
print(S_temp_temp.shape)


: 

: 

In [ ]:
E_temp

In [ ]:
import ucm_mean_pb
ucm_result = ucm_mean_pb.compute_ucm(E_temp, S_temp_temp)
ucm_result

: 

: 

: 

: 

In [4]:
E_upsampled_MATLAB = scipy.io.loadmat('Data_spAffinities/upsampled_E_MATLAB.mat')['upsampled_E_MATLAB']
E_upsampled_MATLAB

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [6]:
print(E_upsampled_MATLAB[:28, 1:7])
# print(E_upsampled_MATLAB[:28, 0:10])

[[0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.09372646 0.09372646 0.09372646]
 [0.         0.         0.         0.09372646 0.         0.        ]
 [0.09372646 0.09372646 0.09372646

In [ ]:
import ucm_mean_pb
ucm_result = ucm_mean_pb.compute_ucm(E_edges_MATLAB, S_temp_temp)

: 

: 

In [12]:
ucm_result.shape

(769, 1025)

In [13]:
# Import U_ucm_mean_pb_MATLAB
U_ucm_mean_pb_MATLAB = scipy.io.loadmat('Data_spAffinities/U_ucm_mean_pb_MATLAB.mat')['U_ucm_mean_pb_MATLAB']
U_ucm_mean_pb_MATLAB.shape

(769, 1025)

In [15]:
print(U_ucm_mean_pb_MATLAB[:10, :5])
print(ucm_result[:10, :5])

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
[[0.         0.         0.         0.         0.        ]
 [0.         0.         0.00941865 0.         0.00282293]
 [0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.00645854 0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]]


In [14]:
# Check if ucm_result is equal to U_ucm_mean_pb_MATLAB
if np.allclose(ucm_result, U_ucm_mean_pb_MATLAB):
    print("ucm_result is equal to U_ucm_mean_pb_MATLAB")


In [47]:
# Print the indices where one of the 2 arrays have a zero and non-zero value
print("Indices where one of the 2 arrays have a zero and non-zero value")
for i in range(len(E_temp)):
    for j in range(len(E_temp[0])):
        if E_temp[i, j] == 0 and E_upsampled_MATLAB[i, j] != 0:
            print(i, j, E_temp[i, j], E_upsampled_MATLAB[i, j])
        elif E_temp[i, j] != 0 and E_upsampled_MATLAB[i, j] == 0:
            print(i, j, E_temp[i, j], E_upsampled_MATLAB[i, j])

Indices where one of the 2 arrays have a zero and non-zero value
10 553 0.049912452697753906 0.0
11 229 0.18540173768997192 0.0
11 230 0.0 0.15276700258255005
11 553 0.049912452697753906 0.0
12 229 0.18540173768997192 0.0
13 558 0.009999999776482582 0.0
13 617 0.05312925577163696 0.0
13 618 0.0 0.05035829544067383
14 617 0.05312925577163696 0.0
15 232 0.13054555654525757 0.0
16 1023 0.0 0.052895426750183105
17 620 0.04928791522979736 0.0
22 531 0.0817253589630127 0.0
23 530 0.0 0.0814942717552185
23 531 0.0817253589630127 0.0
24 309 0.0 0.1534411907196045
25 308 0.16317540407180786 0.0
25 309 0.16317540407180786 0.0
25 534 0.08020651340484619 0.0
27 312 0.07131147384643555 0.0
28 23 0.0 0.16706037521362305
29 22 0.21704864501953125 0.0
29 23 0.21704864501953125 0.0
31 26 0.03218817710876465 0.0
40 1023 0.0 0.09886795282363892
48 487 0.07110720872879028 0.0
49 487 0.07110720872879028 0.0
50 255 0.0 0.0213625431060791
51 254 0.02866220474243164 0.0
51 255 0.02866220474243164 0.0
51 490 0

In [40]:
# Check if E_temp is equal to E_upsampled_MATLAB by a difference threshold  of 1e-2
if np.allclose(E_temp, E_upsampled_MATLAB, atol=10e-2):
    print("E_temp is equal to E_upsampled_MATLAB")
else:
    print("E_temp is not equal to E_upsampled_MATLAB")
    
# Print where the difference is greater than 1e-2 in form of a tuple of arrays
for i in range(E_temp.shape[0]):
    for j in range(E_temp.shape[1]):
        if abs(E_temp[i, j] - E_upsampled_MATLAB[i, j]) > 1e-1:
            print(i, j, E_temp[i, j], E_upsampled_MATLAB[i, j])


E_temp is not equal to E_upsampled_MATLAB
0 350 0.6896791458129883 0.580316424369812
0 800 0.4108849763870239 0.28978025913238525
1 350 0.6896791458129883 0.580316424369812
1 800 0.4108849763870239 0.28978025913238525
2 350 0.6896791458129883 0.580316424369812
2 800 0.4108849763870239 0.28978025913238525
3 350 0.6896791458129883 0.580316424369812
3 800 0.4108849763870239 0.28978025913238525
4 350 0.6896791458129883 0.580316424369812
4 800 0.6342423558235168 0.5050775408744812
5 350 0.6896791458129883 0.580316424369812
5 800 0.6342423558235168 0.5050775408744812
6 350 0.6896791458129883 0.580316424369812
6 800 0.6342423558235168 0.5050775408744812
7 350 0.6896791458129883 0.580316424369812
7 800 0.6342423558235168 0.5050775408744812
8 154 0.30098384618759155 0.17862707376480103
8 350 0.6896791458129883 0.580316424369812
8 800 0.6342423558235168 0.5050775408744812
8 801 0.6342423558235168 0.5050775408744812
8 802 0.6342423558235168 0.5050775408744812
9 154 0.30098384618759155 0.178627073